---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
from scipy.stats import ttest_ind
import datetime as dt
import time as tm
from scipy import stats


In [4]:
# load homes
all_homes = pd.read_csv("City_Zhvi_AllHomes.csv")
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    uni_towns = pd.read_csv("university_towns.txt",delimiter="\n",  header = None)
    uni_towns = uni_towns.rename(columns= {uni_towns.columns[0]:"State/reg"})
    uni_towns["IsState"] = list(map(lambda x: x.find("[edit]")!=-1,uni_towns["State/reg"]))
    uni_towns["State"] = None
    uni_towns["RegionName"] = None
    uni_towns.loc[uni_towns["IsState"],"State"] = list(map(lambda x: x.split("[")[0], uni_towns.loc[uni_towns["IsState"],"State/reg"]))
    uni_towns.loc[uni_towns["IsState"]==False,"RegionName"] = list(map(lambda x: x.split(" (")[0], uni_towns.loc[uni_towns["IsState"]==False,"State/reg"]))
    uni_towns["State"].ffill(inplace=True)
    uni_towns.drop(uni_towns.index[uni_towns["IsState"]==True], inplace=True)
    uni_towns.reset_index(inplace=True, drop=True)
    
    return  uni_towns[["State","RegionName"]]

# get_list_of_university_towns()

In [7]:
gdp_lev = pd.read_excel("gdplev.xls",skiprows=5)
gdp_lev.drop(gdp_lev.index[0:2],inplace=True)
gdp_lev.reset_index(inplace=True)
gdp_lev = (gdp_lev.drop(["index",'Unnamed: 0', 'GDP in billions of current dollars','GDP in billions of chained 2009 dollars', 
                       'Unnamed: 3', 'GDP in billions of current dollars.1','Unnamed: 7'],axis=1))
gdp_lev.rename(columns={"Unnamed: 4" : "YearQ", "GDP in billions of chained 2009 dollars.1":"GDP"},inplace=True)
good_ind = gdp_lev.index[gdp_lev["YearQ"]=="2000q1"][0]
gdp_lev = gdp_lev.iloc[gdp_lev.index[gdp_lev["YearQ"]=="2000q1"][0]:,:]
gdp_lev.reset_index(drop=True,inplace=True)


In [8]:
def find_init_fin_indexes():
    GDP_info = gdp_lev["GDP"].copy()

    first_diff = np.diff(GDP_info)
    second_diff = np.roll(first_diff,-1)
    init_aux = np.where(((first_diff<0) & (second_diff<0)))[0].astype(int)
    fin_aux = np.where(((first_diff>0) & (second_diff>0)))[0].astype(int)
    fin_aux = fin_aux + 2 
    rec_init = [init_aux[0]]
    good_elements = np.where(fin_aux>=init_aux[0])
    fin_aux = fin_aux[good_elements]
    rec_fin = [fin_aux[0]]

    while rec_fin[-1] <= init_aux[-1]:
        recc = np.where((init_aux - rec_fin[-1]) > 0)
        rec_init.append(init_aux[recc[0][0]])
        finn = np.where((fin_aux - rec_init[-1]) > 0)
        rec_fin.append(fin_aux[finn[0][0]])
    return rec_init, rec_fin
rec_init, rec_fin = find_init_fin_indexes()


In [9]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    init_indexes = find_init_fin_indexes()[0]
    
    return gdp_lev.loc[init_indexes, "YearQ"]

get_recession_start()

33    2008q2
Name: YearQ, dtype: object

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    fin_indexes = find_init_fin_indexes()[1]
    return gdp_lev.loc[fin_indexes, "YearQ"]
get_recession_end()

39    2009q4
Name: YearQ, dtype: object

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP_info = gdp_lev["GDP"].copy()
    init_indexes, fin_indexes = find_init_fin_indexes()
    bottom_indexes = np.zeros(np.array(init_indexes).shape)
    for period in range(len(bottom_indexes)):
        bottom_indexes[period] =  GDP_info[init_indexes[period]:fin_indexes[period]].idxmin()
    return gdp_lev.loc[bottom_indexes,"YearQ"]
get_recession_bottom()

37.0    2009q2
Name: YearQ, dtype: object

In [12]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.'''
    

    all_homes = pd.read_csv("City_Zhvi_AllHomes.csv")
    all_homes.set_index(["State","RegionName"],inplace=True)
    years_to_keep = ["{}-{}".format(i,str(j).zfill(2)) for i in range(2000,2017) for j in range(1,13)]
    all_homes = all_homes.loc[:,years_to_keep[:-4]]
    new_col_names = np.repeat(["{}q{}".format(i,j) for i in range(2000, 2017) for j in range(1,5)],3)
    all_homes = all_homes.rename(columns = dict(zip(all_homes.columns[:], new_col_names[:-4])))
    all_homes = all_homes.groupby(by=all_homes.columns, axis=1).mean()
    all_homes.sort_index(inplace=True)
    return all_homes

only_useful_homes = convert_housing_data_to_quarters()
# only_useful_homes

In [13]:
def get_uni_town_acro_merged():
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    states_acronyms = pd.DataFrame(states,index=["FullName"]).T
    states_acronyms
    states_acronyms["State"] = states_acronyms.index
    states_acronyms.set_index("FullName",inplace=True)
    states_acronyms.sort_index(inplace=True)
    uni_towns = get_list_of_university_towns()
    uni_towns.set_index("State",inplace=True)
    uni_towns.index.rename("FulName",inplace=True)
    uni_towns.sort_index(inplace=True)
    uni_towns_merged_acro = pd.merge(uni_towns,states_acronyms,how="right",left_index=True, right_index=True)
    uni_towns_merged_acro.set_index(["State","RegionName"],inplace=True)
    uni_towns_merged_acro["isUniTown"]=True
    uni_towns_merged_acro.sort_values(by="State",inplace=True)
    return uni_towns_merged_acro
    
uni_towns_merged_acro = get_uni_town_acro_merged()
# uni_towns_merged_acro

In [14]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    only_useful_homes = convert_housing_data_to_quarters()
    recession_start =  list((set(get_recession_start()).intersection(set(only_useful_homes.columns))))
    recession_bottom = list(set(get_recession_bottom()).intersection(set(only_useful_homes.columns)))
    only_useful_homes = only_useful_homes.loc[:,recession_start +recession_bottom ]
    only_useful_homes["RatioStartBottom"] = only_useful_homes.apply(lambda x : x[recession_start[0]]/x[recession_bottom[0]],axis =1)
    ratioStartBottom = only_useful_homes.sort_index()
    
    uni_towns_merged_acro = get_uni_town_acro_merged()
    merged_ratio_uni = pd.merge(ratioStartBottom,uni_towns_merged_acro,how="outer",left_on =["State","RegionName"], right_on=["State","RegionName"])
    merged_ratio_uni.sort_values(by=["State","RegionName"],inplace=True)
    merged_ratio_uni = merged_ratio_uni.loc[~np.isnan(merged_ratio_uni["RatioStartBottom"])]

    merged_ratio_uni["isUniTown"].fillna(False,inplace=True)

    dataset_unitowns = merged_ratio_uni.loc[merged_ratio_uni["isUniTown"]==True,"RatioStartBottom"]
    dataset_nonunitowns = merged_ratio_uni.loc[merged_ratio_uni["isUniTown"] == False,"RatioStartBottom"]

    
    ttest_stat, tt_pval = stats.ttest_ind(dataset_nonunitowns, dataset_unitowns)
    alpha = 0.01
    # much larger p than alpha - > cannot reject null hypothesis
    different = (tt_pval < alpha)
    
    better = "university town"
    if dataset_nonunitowns.mean()  > dataset_unitowns.mean():
        better = "non-university town"
      
    return (different, tt_pval, better)

different, p, better = run_ttest()
(different, p, better)


(True, 0.002724063704761164, 'non-university town')